# Data cleanup for modeling through SAT minimums

This notebook is about preparing data to create a model to predict earnings ten years after college graduation based on the college's admissions minumum SAT score requirement.

## Import the datafiles and clean and merge as needed

In [48]:
# For data viewing and manipulation
import pandas as pd

In [49]:
# Read data from csv's into dataframes
original_data = pd.read_csv('Data/Most-Recent-Cohorts-Treasury-Elements.csv')
subset_data = pd.read_csv('Data/datasubset.csv')

In [50]:
# List the dataframes' columns
original_col = original_data.columns
subset_col = subset_data.columns

print(original_col)
print(subset_col)

Index(['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'COUNT_ED', 'AGEGE24',
       'PCT_WHITE', 'PCT_BLACK', 'PCT_ASIAN', 'PCT_HISPANIC', 'PCT_BA',
       'PCT_GRAD_PROF', 'PCT_BORN_US', 'MEDIAN_HH_INC', 'POVERTY_RATE',
       'UNEMP_RATE', 'LN_MEDIAN_HH_INC', 'COUNT_NWNE_P10', 'COUNT_WNE_P10',
       'MN_EARN_WNE_P10', 'MD_EARN_WNE_P10', 'PCT10_EARN_WNE_P10',
       'PCT25_EARN_WNE_P10', 'PCT75_EARN_WNE_P10', 'PCT90_EARN_WNE_P10',
       'SD_EARN_WNE_P10', 'COUNT_WNE_INC1_P10', 'COUNT_WNE_INC2_P10',
       'COUNT_WNE_INC3_P10', 'COUNT_WNE_INDEP0_INC1_P10',
       'COUNT_WNE_INDEP0_P10', 'COUNT_WNE_INDEP1_P10', 'COUNT_WNE_MALE0_P10',
       'COUNT_WNE_MALE1_P10', 'GT_25K_P10', 'MN_EARN_WNE_INC1_P10',
       'MN_EARN_WNE_INC2_P10', 'MN_EARN_WNE_INC3_P10',
       'MN_EARN_WNE_INDEP0_INC1_P10', 'MN_EARN_WNE_INDEP0_P10',
       'MN_EARN_WNE_INDEP1_P10', 'MN_EARN_WNE_MALE0_P10',
       'MN_EARN_WNE_MALE1_P10', 'COUNT_NWNE_P6', 'COUNT_WNE_P6',
       'MN_EARN_WNE_P6', 'MD_EARN_WNE_P6', 'PCT10_EARN_

In [51]:
# Extract the columns needed for the analysis
earnings_df = original_data[['UNITID', 'MN_EARN_WNE_P10', 'MD_EARN_WNE_P10']]

# Rename those columns and set the index to UNITID
earnings_df.rename(columns={'MN_EARN_WNE_P10':'mean_earnings_after10', 
                            'MD_EARN_WNE_P10':'median_earnings_after10'}, inplace=True)

earnings_df.set_index('UNITID', inplace=True)
earnings_df.head()

,mean_earnings_after10,median_earnings_after10
UNITID,,
100654,35500,31000
100663,48400,41200
100690,47600,39600
100706,52000,46700
100724,30600,27700


In [52]:
# Extract the columns needed for the analysis and rest the index to UNITID
filtered_data = subset_data[['UNITID',
                              'Sat_Mp_Critical_Reading',
                              'Sat_Mp_Math']]
filtered_data.set_index('UNITID', inplace=True)
filtered_data.head()

,Sat_Mp_Critical_Reading,Sat_Mp_Math
UNITID,,
100654,425.0,420.0
100663,560.0,575.0
100690,NaN,NaN
100706,590.0,610.0
100724,415.0,410.0


In [53]:
# Merge the two dataframs on UNITID
model_data = earnings_df.merge(filtered_data, on='UNITID')
model_data.head()

,mean_earnings_after10,median_earnings_after10,Sat_Mp_Critical_Reading,Sat_Mp_Math
UNITID,,,,
100654,35500,31000,425.0,420.0
100663,48400,41200,560.0,575.0
100690,47600,39600,NaN,NaN
100706,52000,46700,590.0,610.0
100724,30600,27700,415.0,410.0


In [58]:
# Get a count of what's in there after the NaN drop
print('Counts before na drop:', '\n'*2, model_data.count(), '\n')

# Drop any rows with with NaN values
model_data.dropna(inplace=True)

# Get a count of what's in there after the NaN drop
print('Counts after NaN drop:', '\n'*2, model_data.count())

model_data.head()

Counts before na drop: 

 mean_earnings_after10      1218
median_earnings_after10    1218
Sat_Mp_Critical_Reading    1218
Sat_Mp_Math                1218
dtype: int64 

Counts after NaN drop: 

 mean_earnings_after10      1218
median_earnings_after10    1218
Sat_Mp_Critical_Reading    1218
Sat_Mp_Math                1218
dtype: int64


,mean_earnings_after10,median_earnings_after10,Sat_Mp_Critical_Reading,Sat_Mp_Math
UNITID,,,,
100654,35500,31000,425.0,420.0
100663,48400,41200,560.0,575.0
100706,52000,46700,590.0,610.0
100724,30600,27700,415.0,410.0
100751,51600,44500,550.0,555.0


So it looks like there are almost 5000 colleges that do not have a minumum SAT admission standard.

In [59]:
# Save the dataframe as csv file for the machine learning part
model_data.to_csv('Data/data_for_SAT_model.csv')